* 分类：适用于离散型数据，预测输入数据具体是什么类别
* 回归：适用于连续性数据，通过之前的数据预测未来的数据
* 有监督：输入的数据有其对应的标签
* 无监督：输入的数据没有标签

In [91]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import jieba
import numpy as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

In [18]:
v = DictVectorizer(sparse=True)
d = [{'foo': 1, 'bar': 2},
     {'foo':3, 'baz':1}]
x = v.fit_transform(d)
print(x)
print(type(x))  # sparse._csr.csr_matrix  sparse矩阵
print(v.get_feature_names_out())  # 输出特征值的名称
print('-' * 30)
print(v.inverse_transform(x))  # 把特征转换成原来的样子
print('-' * 30)
print(v.transform({'foo':4, 'unseen_feature': 3}))

  (0, 0)	2.0
  (0, 2)	1.0
  (1, 1)	1.0
  (1, 2)	3.0
<class 'scipy.sparse._csr.csr_matrix'>
['bar' 'baz' 'foo']
------------------------------
[{'bar': 2.0, 'foo': 1.0}, {'baz': 1.0, 'foo': 3.0}]
------------------------------
  (0, 2)	4.0


In [16]:
def dict_vec():
    """
    字典数据特征抽取
    把字典数据向量化
    :return: None
    """
    # 实例化DictVectorizer对象
    dict1 = DictVectorizer(sparse=False)
    # 将字典转换为ont-hot编码的向量
    data = dict1.fit_transform([{'city': '成都', 'temperature':100},
                                {'city': '上海', 'temperature': 90},
                                {'city':'深圳', 'temperature': 70}])
    print(data)
    print('-' * 30)
    print(dict1.get_feature_names_out())
    print('-' * 30)
    print(dict1.inverse_transform(data))

dict_vec()

[[  0.   1.   0. 100.]
 [  1.   0.   0.  90.]
 [  0.   0.   1.  70.]]
------------------------------
['city=上海' 'city=成都' 'city=深圳' 'temperature']
------------------------------
[{'city=成都': 1.0, 'temperature': 100.0}, {'city=上海': 1.0, 'temperature': 90.0}, {'city=深圳': 1.0, 'temperature': 70.0}]


In [34]:
def count_vec():
    """
    对文本进行特征值化,
    单个汉字单个字母不统计，
    因为单个汉字字母没有意义
    :return:
    """
    # 实例化CountVectorizer对象,用于文本特征
    text_vec =  CountVectorizer()
    res = text_vec.fit_transform(["life is short, i love python",
                                  "i hope like me too",
                                  "good good study",
                                  "day day up"])
    # 把每个词都给分离开了
    print(text_vec.get_feature_names_out())
    print('-' * 30)
    print(res)
    print(type(res))
    print('-' * 30)
    # 输出向量化之后的矩阵，标记每个词出现的次数
    print(res.toarray())
    print('-' * 30)
    print(text_vec.inverse_transform(res))

count_vec()

['day' 'good' 'hope' 'is' 'life' 'like' 'love' 'me' 'python' 'short'
 'study' 'too' 'up']
------------------------------
  (0, 4)	1
  (0, 3)	1
  (0, 9)	1
  (0, 6)	1
  (0, 8)	1
  (1, 2)	1
  (1, 5)	1
  (1, 7)	1
  (1, 11)	1
  (2, 1)	2
  (2, 10)	1
  (3, 0)	2
  (3, 12)	1
<class 'scipy.sparse._csr.csr_matrix'>
------------------------------
[[0 0 0 1 1 0 1 0 1 1 0 0 0]
 [0 0 1 0 0 1 0 1 0 0 0 1 0]
 [0 2 0 0 0 0 0 0 0 0 1 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 1]]
------------------------------
[array(['life', 'is', 'short', 'love', 'python'], dtype='<U6'), array(['hope', 'like', 'me', 'too'], dtype='<U6'), array(['good', 'study'], dtype='<U6'), array(['day', 'up'], dtype='<U6')]


In [38]:
text = CountVectorizer()
data = ["人生苦短，我用python","C++很强大，但是我喜欢python"]
data = text.fit_transform(data)
print(text.get_feature_names_out())
print('-' * 30)
print(data)
print('-' * 30)
# 将parse矩阵转换为ndarray
print(data.toarray())

['人生苦短' '但是我喜欢python' '很强大' '我用python']
------------------------------
  (0, 0)	1
  (0, 3)	1
  (1, 2)	1
  (1, 1)	1
------------------------------
<class 'numpy.ndarray'>


In [60]:
def cut_word():
    """
    jieba分词
    :return:
    """
    con1 = jieba.cut("今天很残酷，明天更残酷，后天很美好，但绝对大部分是死在明天晚上，所以每个人不要放弃今天")

    con2 = jieba.cut("我们看到的从很远星系来的光是在几百万年之前发出的，这样当我们看到宇宙时，我们是在看它的过去。")

    con3 = jieba.cut("如果只用一种方式了解某样事物，你就不会真正了解它。了解事物真正含义的秘密取决于如何将其与我们所了解的事物相联系。")

    print(type(con1)) # generator
    print('-' * 30)
    # 将生成器转换为列表类型
    content1 = list(con1)
    content2 = list(con2)
    content3 = list(con3)
    # print(content1)
    # print(content2)
    # print(content3)
    # print('-' * 30)
    c1 = ' '.join(content1)
    c2 = ' '.join(content2)
    c3 = ' '.join(content3)

    return c1, c2, c3


cut_word()

<class 'generator'>
------------------------------
<class 'str'>


('今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天',
 '我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。',
 '如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。')

In [52]:
def chine_vec():
    """
    中文特征化
    :return:None
    """
    c1, c2, c3 = cut_word()
    cv = CountVectorizer()
    data = cv.fit_transform([c1, c2,c3])
    print(cv.get_feature_names_out())
    print(data.toarray())

chine_vec()

<class 'generator'>
------------------------------
['今天', '很', '残酷', '，', '明天', '更', '残酷', '，', '后天', '很', '美好', '，', '但', '绝对', '大部分', '是', '死', '在', '明天', '晚上', '，', '所以', '每个', '人', '不要', '放弃', '今天']
['我们', '看到', '的', '从', '很', '远', '星系', '来', '的', '光是在', '几百万年', '之前', '发出', '的', '，', '这样', '当', '我们', '看到', '宇宙', '时', '，', '我们', '是', '在', '看', '它', '的', '过去', '。']
['如果', '只用', '一种', '方式', '了解', '某样', '事物', '，', '你', '就', '不会', '真正', '了解', '它', '。', '了解', '事物', '真正', '含义', '的', '秘密', '取决于', '如何', '将', '其', '与', '我们', '所', '了解', '的', '事物', '相', '联系', '。']
------------------------------
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
[[0 0 1 0 0 0 2 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 2 0 1 0 2 1 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 3 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 1 1]
 [1 1 0 0 4 3 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 2 1 0 0

In [67]:
def tf_idf_vec():
    """
    计算TF-IDF值：表示改文件的重要程度
    TF：表示改词出现的次数
    IDF：表示逆文档频率
    :return:
    """
    # 把通过jieba分词之后的字符串用于特征提取
    c1, c2,c3 = cut_word()
    data = [c1,c2,c3]
    # 实例化TfidfVectorizer()对象
    tf = TfidfVectorizer()
    # 向量化
    res = tf.fit_transform(data)
    # 输出特征的具体名称
    print(tf.get_feature_names_out())
    print('-' * 30)
    print(res)
    print(type(res)) # parse矩阵
    # 将parse矩阵转换为ndarray
    print(res.toarray())

tf_idf_vec()

<class 'generator'>
------------------------------
<class 'str'>
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
------------------------------
  (0, 20)	0.2182178902359924
  (0, 2)	0.2182178902359924
  (0, 27)	0.2182178902359924
  (0, 19)	0.2182178902359924
  (0, 24)	0.2182178902359924
  (0, 14)	0.2182178902359924
  (0, 31)	0.2182178902359924
  (0, 32)	0.2182178902359924
  (0, 12)	0.2182178902359924
  (0, 22)	0.4364357804719848
  (0, 26)	0.4364357804719848
  (0, 6)	0.4364357804719848
  (1, 34)	0.24108220270067757
  (1, 17)	0.24108220270067757
  (1, 35)	0.24108220270067757
  (1, 9)	0.24108220270067757
  (1, 3)	0.24108220270067757
  (1, 8)	0.24108220270067757
  (1, 7)	0.24108220270067757
  (1, 23)	0.24108220270067757
  (1, 28)	0.48216440540135513
  (1, 18)	0.5500476874707075
  (2, 33)	0.15698297076974738
  (2, 15)	0.15698297076974738
  (2, 10)	0.15

### 特征处理

In [71]:
def min_max():
    """
    归一化处理
    :return:None
    """
    # X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    min_max = MinMaxScaler(feature_range=(0,1))
    data = [[90,2,10,40],[60,4,15,45],[75,3,13,46]]
    res = min_max.fit_transform(data)
    print(res)


min_max()

[[1.         0.         0.         0.        ]
 [0.         1.         1.         0.83333333]
 [0.5        0.5        0.6        1.        ]]
------------------------------
[[90, 2, 10, 40], [60, 4, 15, 45], [75, 3, 13, 46]]


In [80]:
def stand():
    """
    标准化缩放,均值为0，方差为1
    :return:
    """
    std = StandardScaler()
    data = [[1.,-1.,3.],[2.,4.,2.],[4.,6.,-1.]]
    res = std.fit_transform(data)
    print(res)
    print('-' * 30)
    print(std.mean_) # 均值
    print(std.var_)  # 方差
    print(std.n_samples_seen_)  # 样本数
    print('-' * 50)
    res1 = std.fit_transform([[-1.06904497, -1.35873244,  0.98058068],
                              [-0.26726124,  0.33968311,  0.39223227],
                              [ 1.33630621,  1.01904933, -1.37281295]])
    print(res1)
    print('-' * 50)
    print(std.mean_) # 均值=0
    print(std.var_)  # 方差=1
    print(std.n_samples_seen_)  # 样本数

stand()

[[-1.06904497 -1.35873244  0.98058068]
 [-0.26726124  0.33968311  0.39223227]
 [ 1.33630621  1.01904933 -1.37281295]]
------------------------------
[2.33333333 3.         1.33333333]
[1.55555556 8.66666667 2.88888889]
3
--------------------------------------------------
[[-1.06904497 -1.35873244  0.98058068]
 [-0.26726124  0.33968311  0.39223227]
 [ 1.33630621  1.01904933 -1.37281295]]
--------------------------------------------------
[0. 0. 0.]
[1.         1.         1.00000001]
3


### 缺失值处理

In [85]:
def imput():
    """
    缺失值的处理
    :return: None
    """
    im = SimpleImputer(missing_values=np.nan, strategy='mean')
    data = [[1,2],[np.nan,3],[7,6],[3,2]]
    res = im.fit_transform(data)
    print(res)

imput()

[[1.         2.        ]
 [3.66666667 3.        ]
 [7.         6.        ]
 [3.         2.        ]]


In [90]:
def drop_var():
    """
    删除低方差特征
    :return:
    """
    d_var = VarianceThreshold(threshold=1)
    data = [[0,2,0,3],
            [0,1,4,3],
            [0,1,1,3]]
    res = d_var.fit_transform(data)
    print(res)
    print('-' * 30)
    # 查看剩余的特征的列编号
    print(d_var.get_support())
    print(d_var.get_support(True))

drop_var()

[[0]
 [4]
 [1]]
------------------------------
[False False  True False]
[2]


In [93]:
def pca():
    """
    进行特征降维
    :return:
    """
    # n_component选在在0-1之间
    pca = PCA(n_components=0.9)
    data = [[2,8,4,5],[6,3,0,8],[5,4,9,1]]
    res = pca.fit_transform(data)
    print(res)

pca()

[[ 1.28620952e-15  3.82970843e+00]
 [ 5.74456265e+00 -1.91485422e+00]
 [-5.74456265e+00 -1.91485422e+00]]
